# GigaReflexion
В этом ноутбуке создадим граф, который расширяет уже существующий ноутбук [reflexion](../reflexion_giga/reflexion.ipynb)

Это по сути агент, который итеративно улучшает ответ на вопрос пользователя, с помощью критики,
поиска в интернете ответов на критику и обновления ответа исходя из результатов поиска.

В этом ноутбук есть 4 компонента:
1. [Цепочка, которая отвечает на вопрос пользователя](#Первоначальный-ответ-на-вопрос-пользователя)
2. Цепочка, которая критикует ответ сгенерированный на шаге 1 и предлагает поисковые запросы для решения критики
3. Поиск в интернете в гугле с выгрузкой страниц и поиск релятивных кусков текста с помощью embedding'ов
4. Цепочка, которая обновляет ответ исходя из результатов поиска

В данной реализации генерация ответов занимает 10-20 минут, что не очень подходит для применения в работе, но
внизу напишу свои мысли, как это можно ускорить до 1-3 минуты для применения данного агента в бизнес-кейсах

Далее внизу будет настройка всей цепочки, для просмотра итоговой работы графа — [сюда](#Построение-графа)

## Настройка ключей / LLM

In [ ]:
% pip install chromadb gigachain

In [1]:
import os

os.environ["GOOGLE_API_KEY"] = "<ключ к гугл поиску>"
os.environ["GOOGLE_CSE_ID"] = "<cse id гугл поиску>"
os.environ["OPENAI_API_KEY"] = "<ключ к open ai>"

Здесь можно заметить, что я выношу температуру и top_p, как configurable_fields.
Почему? Потому что, допустим для поиска в интернете используются цепочки перевода, которым нужна температура 0.
А вот остальным цепочкам я ставлю температуру 1 или top_p=0.6, как в 3 цепочке.
То есть мы используем один llm объект, а каждая цепочка может параметры настроить под себя

In [2]:
from langchain_community.chat_models.gigachat import GigaChat
from langchain_core.runnables import ConfigurableField

llm = GigaChat(
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro",
    temperature=1,
    top_p=None
).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM",
    ),
    top_p=ConfigurableField(
        id="llm_top_p",
        name="LLM Top P",
    )
)

## Цепочки
### Первоначальный ответ на вопрос пользователя

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Ты бот эксперт-исследователь. Ты помогаешь пользователю в его исследованиях.
Если тебя просят получить последние новости, то возвращай "NEED_INFO"
Ответь на вопрос.

Вопрос пользователя: "{question}"

Твои действия:
1. Прочитай мой вопрос
2. Подумай как лучше всего ответить на него
3. Дай короткий ответ из 2-4 предложений.

Ответ: """,
        )
    ]
)

answer_chain = prompt | llm


def answer_respond(state):
    return {"current_answer": answer_chain.invoke(state).content, "iteration": state["iteration"] + 1}

#### Тестируем её работу

In [177]:
state = {
    "messages": [],
    "iteration": 1,
    "question": "Как победить глобальное потепление?",
    "current_answer": "",
    "reflection": "",
    "search_queries": [],
    "search_results": {},
    "downloaded_urls": set(),
    "finish": False,
    "lang": "en"
}
new_state = answer_respond(state)
state = {**state, **new_state}
state['current_answer']

'Глобальное потепление - серьезная проблема, требующая комплексного подхода. Необходимо уменьшить выбросы парниковых газов, сократив использование ископаемого топлива и увеличив использование возобновляемых источников энергии. Также важно увеличить энергоэффективность и озеленение городов. Кроме того, необходимо развивать экологическое образование и повышать осведомленность общества о проблеме глобального потепления.'

### Критика текущего ответа из state
*Важно*
Хочу заметить, что ниже очень полезный класс, позволяющий заполнять аргументы функции, если гигачат их пропустил.
А он пока что любит пропускать некоторые аргументы, даже если они указаны как required

In [4]:
from langchain_core.messages import HumanMessage
from langsmith import traceable


class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    @traceable
    def respond(self, state):
        response = []
        messages = state["messages"]
        for attempt in range(5):
            try:
                response = self.runnable.invoke(state)
                self.validator.invoke(response)
                args = response.additional_kwargs["function_call"]["arguments"]
                current_answer = state["current_answer"]
                if current_answer == "NEED_INFO":
                    current_answer = "      "
                return {
                    "messages": messages + [response],
                    "iteration": state["iteration"] + 1,
                    "current_answer": current_answer,
                    **args,
                }
            except Exception as e:
                message = HumanMessage(content=repr(e))
                if response.response_metadata["finish_reason"] != "error":
                    state["messages"] += [message]
        return {
            "messages": messages + [response],
            "iteration": state["iteration"] + 1,
        }

In [5]:
from typing import List

from langchain_core.output_parsers.gigachat_functions import (
    PydanticOutputFunctionsParser,
)
from pydantic.v1 import BaseModel, Field

reflection_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Ты опытный критик.
Как разносторонняя личность, ты все ставишь под сомнение, и критикуешь всё.
Не стесняйся в выражениях и не сдерживай себя.
Текущая дата: {time}

Что ты должен сделать:
1. Прочитать мой ответ
2. Хорошенько подумай над ним
3. Подумай что с ним не так
4. Напиши свою критику
5. Порекомендуй поисковые запросы для решения критики

Вопрос: "{question}"
Мой ответ: "{current_answer}"
{search_results}
""",
        ),
    ]
).partial(time=lambda: "17.03.2014", search_results="")


class ReflectAnswer(BaseModel):
    """Функция критики моего ответа."""

    reflection: str = Field(
        description="Критика моего ответа. Что в нем можно дополнить?"
    )
    search_queries: List[str] = Field(
        description="1–3 поисковых запроса, которые позволят решить критику и дополнить мой текущий ответ"  # noqa
    )


initial_answer_chain = reflection_template | llm.default.bind_tools(
    tools=[ReflectAnswer], tool_choice="ReflectAnswer"
)
validator = PydanticOutputFunctionsParser(pydantic_schema=ReflectAnswer)

reflection_responder = ResponderWithRetries(
    runnable=initial_answer_chain, validator=validator
)


def reflect_respond(state):
    return reflection_responder.respond(state)


#### Тестируем её работу

In [180]:
state = {**state, **reflection_responder.respond(state)}
state["messages"][-1].additional_kwargs["function_call"]["arguments"]

{'reflection': 'Твой ответ слишком общий и не предлагает конкретных решений. Как насчет предложения мер по адаптации к изменению климата?',
 'search_queries': ['меры по адаптации к изменению климата',
  'конкретные действия для борьбы с глобальным потеплением']}

### Поиск в интернете
В данном блоке мы ищем информацию в гугле, с помощью поисковых запросов, которые мы получили из предыдущего компонента.
Также тут переводятся поисковые запросы на английский язык, для получения ещё большего количества информации, и
результаты поиска переводятся обратно на русский, чтобы можно было использовать найденную информацию для лучшего
формирования ответа. (тут можно конечно по-экспериментировать)
Перевод включается если в state, мы указали lang=en
Хочу заметить, что мы в state графа, храним уже скачанные страницы,
поэтому мы не используем дублирующую информацию для расширения ответа.
Но возможно стоит хранить всю информацию, выгруженную с веб-страниц, просто удаляя куски, которые использовали в векторной БД

In [6]:
from prompts import translate_to_english_prompt, translate_to_russian_prompt

translator_to_english = translate_to_english_prompt | llm | (lambda message: message.content)
translator_russian = (
    translate_to_russian_prompt | llm | (lambda message: message.content)
)

In [7]:
import re

from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.vectorstores.chroma import Chroma
from web_retriever import GigaWebResearchRetriever


def search(query, downloaded_urls):
    """
    Поиск информации в гугле по поисковому запросу, с выгрузкой страниц
    и поиска релятивных кусков с помощью embedding'ов
    """
    vectorstore_public = Chroma(
        collection_name="web",
        embedding_function=OpenAIEmbeddings(),
    )

    search = GoogleSearchAPIWrapper()

    # Initialize
    web_retriever = GigaWebResearchRetriever.from_llm(
        vectorstore=vectorstore_public,
        llm=llm,
        search=search,
        num_search_results=10,
        verify_ssl=False,
    )
    web_retriever.retrieve_kwargs = {"k": 4}
    web_retriever.download_num = 5
    web_retriever.url_database = list(downloaded_urls)
    docs = web_retriever.get_relevant_documents(query)
    vectorstore_public.delete_collection()
    return docs


def execute_tools(state):
    """Ищем по поисковым запросом нужную информацию"""
    search_queries = state["search_queries"]
    # Переводим запросы на английский, если в state язык поиска указан "en"
    if state["lang"] == "en":
        translated = translator_to_english.with_config(llm_temperature=0.01).batch(
            [{"text": q} for q in search_queries]
        )
    else:
        translated = search_queries
    results = []
    downloaded_urls = state["downloaded_urls"]
    for i, q in enumerate(translated):
        docs = search(q, downloaded_urls)
        downloaded_urls |= {doc.metadata["source"] for doc in docs}
        results.append(docs)
    # Исключаем уже найденные ранее куски информации
    vectorstore = Chroma(
        collection_name="web1",
        embedding_function=OpenAIEmbeddings(),
    )
    search_results = []
    for result, search_query in zip(results, search_queries):
        for doc in result:
            similar = vectorstore.similarity_search_with_score(doc.page_content)
            f = similar[0][1] if len(similar) > 0 else 1
            if f > 0.01:
                doc.metadata["query"] = search_query
                vectorstore.add_documents(documents=[doc])
                search_results.append(doc)
    # Переводим информацию обратно на русский, если искали на английском
    if state["lang"] == "en":
        translated = translator_russian.with_config(llm_temperature=0.01).batch(
            [doc.page_content for doc in search_results]
        )
        for trans, doc in zip(translated, search_results):
            doc.page_content = trans
    # Формируем словарь запросов, которые мы нашли
    search_map = {}
    for result in search_results:
        search_map.setdefault(result.metadata["query"], [])
        search_map[result.metadata["query"]].append(result)
    # Формируем словарь результатов поиска
    result_map = {}
    for query in search_map.keys():
        result_string = f"Поисковой запрос: {query}\n\n"
        search_items = []
        char_count = len(result_string)
        for item in search_map[query]:
            content = re.sub("\n+", "\n", item.page_content)
            search_item = f"""Заголовок: "{item.metadata['title']}"\nСодержание: "{content}"\nСсылка: "{item.metadata['source']}"\n"""
            char_count += len(search_item) + 1
            search_items.append(search_item)
        result_string += "\n".join(search_items) + "\n\n\n"
        result_map[query] = result_string
    return {"search_results": result_map, "downloaded_urls": downloaded_urls}

#### Тестируем работу

In [8]:
import warnings

# Убираем warnings, которые возникают из-за того, что мы грузим страницы
# не через SSL (чтобы ноутбук не расплылся)
warnings.filterwarnings('ignore')

In [186]:
state = {**state, **execute_tools(state)}
print(state['search_results']["меры по адаптации к изменению климата"].strip())

Fetching pages: 100%|##########| 5/5 [00:01<00:00,  2.65it/s]


Поисковой запрос: меры по адаптации к изменению климата

Заголовок: "What is climate change adaptation? - Grantham Research Institute on climate change and the environment"
Содержание: "Какие существуют меры адаптации?
Меры адаптации могут быть запланированы заранее или приняты реагированием. Страны с низким уровнем дохода и определенные группы населения в них, как правило, более уязвимы к рискам, связанным с изменением климата, чем развитые страны, и некоторые меры адаптации - такие как увеличение доступа к образовательным и медицинским учреждениям - будут совпадать с существующими программами развития. Существует также значительное совпадение между мерами по адаптации к изменению климата и снижению рисков, связанных с стихийными бедствиями (СРБ), и политиками - в частности, Рамочной программой Сендая по снижению рисков, связанных со стихийными бедствиями.
Однако адаптация выходит за рамки существующих подходов к развитию и включает меры по устранению дополнительных рисков, непосредст

### Цепочка обновления ответа
Здесь мы по каждому поисковому запросу генерируем мини ответ.
После этого объединяем эти ответы со старым ответом и подменяем current_answer на новый ответ.

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Ты — expert researcher.

1. Прочитай ответ
2. Внимательно прочитай информацию из интернета
3. Напиши 2-3 предложения, отвечающих на критику, используя важную информацию из интернета
""",  # noqa
        ),
        (
            "user",
            """Вопрос пользователя: "{question}"
Ответ: "{current_answer}"
Критика: "{reflection}"
Информация из интернета: "{search_results}"
Ответ на критику: """,
        ),
    ]
)
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Ты — expert researcher.

1. Прочитай ответ
2. Внимательно прочитай информацию из интернета
3. Допиши ответ, используя информацию из интернета
""",  # noqa
        ),
        (
            "user",
            """Вопрос пользователя: "{question}"
Ответ: "{current_answer}"
Критика: "{reflection}"
Информация из интернета: "{search_results}"
Обновленный ответ: """,
        ),
    ]
)

revise_chain = prompt | llm.with_retry()
revise_chain2 = prompt2 | llm


def revise_respond(state):
    answers = revise_chain.with_config(llm_top_p=0.8, llm_temperature=None).batch(
        [
            {
                **state,
                **{
                    "search_results": result.strip(),
                    "current_answer": state["current_answer"],
                    "reflection": query + "?",
                },
            }
            for query, result in state["search_results"].items()
        ]
    )
    answers = [
        re.sub(
            r"^ответ на критику:?\s?",
            "",
            answer.content,
            0,
            re.MULTILINE | re.IGNORECASE,
        ).strip("\"'")
        for answer in answers
    ]
    new_answer = ""
    retries = 0
    # Проверяем насколько сильно модель обрезала ответ
    while (len(new_answer) + 1) / (len(state["current_answer"]) + 1) < 0.75:
        print("combine answer")
        retries += 1
        new_answer = revise_chain2.with_config(llm_top_p=0.6, llm_temperature=None).invoke(
            {**state, **{"search_results": "\n".join(answers)}}
        ).content
        # Если модель постоянно уменьшает ответ отдаем на финиш
        if retries > 10:
            return {"finish": True, "current_answer": state["current_answer"]}
    return {"current_answer": new_answer, "iteration": state["iteration"] + 1}

#### Тестируем работу

In [188]:
state = {**state, **revise_respond(state)}
print(state["current_answer"])

combine answer
Для победы над глобальным потеплением требуется комплексный подход. Важными мерами являются сокращение выбросов парниковых газов путем снижения использования ископаемого топлива и увеличения использования возобновляемых источников энергии, повышение энергоэффективности и озеленение городов. Однако, помимо этого, необходимо также принимать меры по адаптации к уже запущенным климатическим изменениям. Это может включать внесение масштабных изменений в инфраструктуру, учет будущих рисков при проектировании текущей инфраструктуры, а также применение природных подходов, таких как сохранение и восстановление лесов, болот и мангровых зарослей.

В индивидуальном плане каждый человек может внести свой вклад в борьбу с глобальным потеплением. Например, можно использовать свой голос в качестве потребителя, клиента, члена избирателей и активного гражданина, присоединиться к социальным движениям или кампаниям, связанным с экологической деятельностью, перейти на растительную диету, исп

# Построение графа
Соединяем все компоненты

In [10]:
from typing import Dict, List, Literal, Sequence, Set, TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    iteration: int
    question: str
    current_answer: str
    reflection: str
    search_queries: List[str]
    search_results: Dict[str, str]
    downloaded_urls: Set[str]
    finish: bool
    lang: Literal["ru", "en"]


MAX_ITERATIONS = 8
builder = StateGraph(AgentState)
builder.add_node("draft", answer_respond)
builder.add_node("reflect", reflect_respond)
builder.add_node("search", execute_tools)
builder.add_node("revise", revise_respond)

builder.add_edge("draft", "reflect")
builder.add_edge("reflect", "search")
builder.add_edge("search", "revise")

# Define looping logic:


def event_loop(state) -> str:
    # in our case, we'll just stop after N plans
    if state["finish"] or state["iteration"] > MAX_ITERATIONS:
        return END
    return "reflect"


builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")
graph = builder.compile()

# Пример работы
Проверяем, как улучшается ответ с 5 итераций

In [14]:
question = "Выиграет ли Дональд Трамп на выборах 2024?"

events = graph.stream(
    {
        "messages": [],
        "iteration": 0,
        "question": question,
        "current_answer": "",
        "reflection": "",
        "search_queries": [],
        "search_results": {},
        "downloaded_urls": set(),
        "finish": False,
        "lang": "en"
    },
    {"recursion_limit": 100},
)

In [15]:
# Удаляем старые данные из хрома, которые возможно сохранились в ноутбуке
Chroma(
    collection_name="web",
    embedding_function=OpenAIEmbeddings(),
).delete_collection()
Chroma(
    collection_name="web1",
    embedding_function=OpenAIEmbeddings(),
).delete_collection()
# Запускаем граф
for i, step in enumerate(events):
    node, output = next(iter(step.items()))
    print(f"## {i+1}. {node}")
    if node == "draft":
        print(f"Ответ: {output['current_answer']}")
    if node == "reflect":
        print(f"Критика: {output['reflection']}")
        print(f"Что будем искать: {output['search_queries']}")
    if node == "search":
        print("Произошел поиск")
    if node == "revise":
        print(f"Ответ: {output['current_answer']}")

    print("---")


## 1. draft
Ответ: NEED_INFO
---


Giga generation stopped with reason: function_call


## 2. reflect
Критика: Твой ответ не содержит никакой информации или мнения о том, выиграет ли Дональд Трамп на выборах 2024 года.
Что будем искать: ['выборы США 2024', 'Дональд Трамп прогнозы выборов']
---


Fetching pages: 100%|##########| 5/5 [00:03<00:00,  1.41it/s]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


## 3. search
Произошел поиск
---
combine answer
## 4. revise
Ответ: Выиграет ли Дональд Трамп на выборах 2024 года? Ответ на этот вопрос зависит от многих факторов. Согласно данным из интернета, Дональд Трамп имеет высокую поддержку среди республиканцев - 67%. Исторически сильные лидеры обычно выигрывают номинации своих партий. Однако, в ключевых штатах, таких как Айова и Нью-Гэмпшир, поддержка Трампа ниже 50%, что оставляет возможность для других кандидатов получить поддержку. Опросы показывают, что потенциальный матч между Трампом и Байденом будет очень близким, и точный исход остается неизвестным из-за погрешности опросов.
---


...

## 9. search
Произошел поиск
---
combine answer
## 10. revise
Ответ: Выиграет ли Дональд Трамп на выборах 2024 года? Ответ на этот вопрос зависит от множества факторов. Согласно последним данным, Дональд Трамп продолжает сохранять высокую поддержку среди республиканцев - около 67%. Однако, в ключевых штатах, таких как Айова и Нью-Гэмпшир, его поддержка ниже 50%, что оставляет возможность для других кандидатов получить поддержку. Потенциальный матч между Трампом и Байденом, согласно опросам, будет очень близким, и точный исход остается неизвестным из-за погрешности опросов.

Существует множество факторов, которые могут повлиять на исход выборов 2024 года. Это включает уровень поддержки Дональда Трампа среди республиканцев, его популярность в ключевых штатах, таких как Айова и Нью-Гэмпшир, и возможное влияние решения Верховного суда США относительно абортов на молодых латиноамериканских избирателей. Также важно учитывать, как дезинформация, распространяемая через социальные сети, может в

Giga generation stopped with reason: function_call


## 11. reflect
Критика: Твой ответ содержит много полезной информации, но он слишком общий и теоретический. Он не учитывает конкретные факторы, которые могут повлиять на исход выборов 2024 года. Например, недавние скандалы, связанные с Трампом, или изменение общественного мнения о нем. Также было бы полезно рассмотреть, как новые технологии и социальные медиа могут повлиять на результаты выборов.
Что будем искать: ['Последние скандалы, связанные с Трампом', 'Изменение общественного мнения о Трампе', 'Влияние новых технологий и социальных медиа на выборы']
---


Fetching pages:  20%|##        | 1/5 [00:01<00:07,  1.88s/it]Error fetching https://www.princeton.edu/~fujiwara/papers/SocialMediaAndElections.pdf with attempt 1/2: . Retrying...
Error fetching https://www.accc.gov.au/system/files/ACCC+commissioned+report+-+The+impact+of+digital+platforms+on+news+and+journalistic+content,+Centre+for+Media+Transition+(2).pdf with attempt 1/2: . Retrying...
Error fetching https://www.princeton.edu/~fujiwara/papers/SocialMediaAndElections.pdf after 2 retries.
Fetching pages: 100%|##########| 5/5 [00:11<00:00,  2.34s/it]


## 12. search
Произошел поиск
---
combine answer
## 13. revise
Ответ: Выиграет ли Дональд Трамп на выборах 2024 года? Ответ на этот вопрос зависит от множества факторов. Согласно последним данным, Дональд Трамп продолжает сохранять высокую поддержку среди республиканцев - около 67%. Однако, в ключевых штатах, таких как Айова и Нью-Гэмпшир, его поддержка ниже 50%, что оставляет возможность для других кандидатов получить поддержку. Потенциальный матч между Трампом и Байденом, согласно опросам, будет очень близким, и точный исход остается неизвестным из-за погрешности опросов.

Существует множество факторов, которые могут повлиять на исход выборов 2024 года. Это включает уровень поддержки Дональда Трампа среди республиканцев, его популярность в ключевых штатах, таких как Айова и Нью-Гэмпшир, и возможное влияние решения Верховного суда США относительно абортов на молодых латиноамериканских избирателей. Также важно учитывать, как дезинформация, распространяемая через социальные сети, может 

In [16]:
print(f"{step[END]['current_answer']}")

Выиграет ли Дональд Трамп на выборах 2024 года? Ответ на этот вопрос зависит от множества факторов. Согласно последним данным, Дональд Трамп продолжает сохранять высокую поддержку среди республиканцев - около 67%. Однако, в ключевых штатах, таких как Айова и Нью-Гэмпшир, его поддержка ниже 50%, что оставляет возможность для других кандидатов получить поддержку. Потенциальный матч между Трампом и Байденом, согласно опросам, будет очень близким, и точный исход остается неизвестным из-за погрешности опросов.

Существует множество факторов, которые могут повлиять на исход выборов 2024 года. Это включает уровень поддержки Дональда Трампа среди республиканцев, его популярность в ключевых штатах, таких как Айова и Нью-Гэмпшир, и возможное влияние решения Верховного суда США относительно абортов на молодых латиноамериканских избирателей. Также важно учитывать, как дезинформация, распространяемая через социальные сети, может влиять на политические настроения и поведение при голосовании латиноам

# Выводы
Из этого графа, можно построить ответы на вопросы по уже заготовленной векторной БД. Большую часть времени для формирования ответа занимает выгрузка страниц из гугла, их перевод. Если бы это делала rag-система, то время формирования ответа бы ускорилось.
+ Можно использовать более легкие версии гигачата для формирования ответа